<a href="https://colab.research.google.com/github/mycheicky/Advance-Numerical-Methods-for-PDE/blob/master/NextGateTech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
# 1) Check the variables from the data 
df = pd.read_csv('ngt_software_engineer_test_example_data (clean columns).csv', header=0, sep=',', decimal='.')

In [ ]:
# 2) The checking of the data
print(df.shape[0], ' rows')
print(df.shape[1], ' columns ')
print('\nList of variables: ')
for col in df.columns:
    print(col)

# List of columns having NaN values
print('\nTypes of variables')
print(df.dtypes.value_counts(), '\n')
print('Data has ', df.isna().sum().sum(), ' number of NaN', '\n')
print('NAN per columns')
for col in df:
  if df[col].isna().sum() > 0:
    print(col, ' has ', df[col].isna().sum(), ' NaN')

In [ ]:
# 3) Calculate the correlation of 11 instruments 'Nav_Per_Share'
fund_code = df['Subfund_Code'].unique() # Here I notice that I've 3 subfunds so I's going to creat 3
val = []

# ==> Check all the graphs of each instrument to preview if there are some correlations between some
for sub in fund_code:
  fund = df[df['Subfund_Code'] == sub ]
  clas = fund['Share_Class_Code'].unique()

  print()
  print('Sub Fund: ', sub, ' | Currency of the sub fund: ', fund['Subfund_CCY'].unique())

  for cl in clas:
    data = fund[fund['Share_Class_Code'] == cl]

    # Preparing the correlation data
    val.append(pd.DataFrame({'Date': data['Valuation_Date'], str(sub) + str(cl): data['NAV_Per_Share']}))

    print()
    print('Share class: ', str(cl), ' | NAV Currency: ', data['CCY_NAV_share'].unique())
    
    # Graph of each instrument per sub fund
    plt.figure(figsize=(15, 10))
    plt.plot( data['NAV_Per_Share'], label=str(sub) + str(cl))
    plt.legend()
    plt.show()

In [ ]:
# ==> The computing of the correlation between the 11 instruments
#     The building of the correlation matrix data based on the Valuation_Date and removing the missing data (using inner merge)
instr = val[0]
for i in range(1, len(val)):
  instr = instr.merge(val[i], on='Date')

#    The correlation computing
instr.corr()

,1A,1B,1C,1D,2A,2B,2C,2D,2E,3A,3B
1A,1.000000,0.993017,0.997665,0.997716,0.894131,0.965188,0.954956,0.968039,0.959560,-0.815522,-0.815356
1B,0.993017,1.000000,0.995144,0.995176,0.857470,0.977551,0.968342,0.977274,0.971292,-0.757360,-0.757213
1C,0.997665,0.995144,1.000000,0.999997,0.871670,0.972747,0.963825,0.974604,0.967455,-0.779614,-0.779416
1D,0.997716,0.995176,0.999997,1.000000,0.871961,0.972768,0.963848,0.974644,0.967484,-0.779930,-0.779732
2A,0.894131,0.857470,0.871670,0.871961,1.000000,0.828887,0.828461,0.848386,0.837385,-0.912778,-0.912718
2B,0.965188,0.977551,0.972747,0.972768,0.828887,1.000000,0.990821,0.992359,0.991409,-0.677415,-0.677254
2C,0.954956,0.968342,0.963825,0.963848,0.828461,0.990821,1.000000,0.998675,0.999829,-0.654143,-0.654028
2D,0.968039,0.977274,0.974604,0.974644,0.848386,0.992359,0.998675,1.000000,0.999388,-0.687824,-0.687693
2E,0.959560,0.971292,0.967455,0.967484,0.837385,0.991409,0.999829,0.999388,1.000000,-0.667414,-0.667299
3A,-0.815522,-0.757360,-0.779614,-0.779930,-0.912778,-0.677415,-0.654143,-0.687824,-0.667414,1.000000,0.999998


In [ ]:
#    Convert the correlation matrix to dict
corr_dict = instr.corr().to_dict()

In [22]:
# 4) Firebase management

# To get acces to my Firebase project
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Fetch the service account key JSON file contents
cred = credentials.Certificate('/content/ngtdevtest-firebase-adminsdk-nk5uo-8711d47d79.json')

# Initialize the app with a custom auth variable, limiting the server's access
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://ngtdevtest-default-rtdb.firebaseio.com/',
    'databaseAuthVariableOverride': {
        'uid': 'service-account'
    }
}, name="ngtqdtApp")

In [26]:
# As an admin, the app has access to read and write all data, regradless of Security Rules
ref = db.reference('/')
ref.child('thekeyresults').set(corr_dict)

In [27]:
print(ref.get())

{'thekeyresults': {'1A': {'1A': 1.0, '1B': 0.993017338448721, '1C': 0.9976651242220277, '1D': 0.9977164089934377, '2A': 0.8941310336113583, '2B': 0.9651881502812986, '2C': 0.9549558108488158, '2D': 0.9680389194097627, '2E': 0.9595595807288593, '3A': -0.8155216411762898, '3B': -0.8153561195659342}, '1B': {'1A': 0.993017338448721, '1B': 1.0, '1C': 0.9951441906160894, '1D': 0.9951757758696038, '2A': 0.8574700821754557, '2B': 0.9775509855418766, '2C': 0.9683420342676197, '2D': 0.9772737167202332, '2E': 0.9712916727741885, '3A': -0.7573599785026255, '3B': -0.7572133906734726}, '1C': {'1A': 0.9976651242220277, '1B': 0.9951441906160894, '1C': 1.0, '1D': 0.9999969098301892, '2A': 0.8716696352416526, '2B': 0.9727465520711558, '2C': 0.9638247564576181, '2D': 0.9746042784783738, '2E': 0.9674546399189106, '3A': -0.7796140360528588, '3B': -0.7794159712203539}, '1D': {'1A': 0.9977164089934377, '1B': 0.9951757758696038, '1C': 0.9999969098301892, '1D': 1.0, '2A': 0.8719610189914749, '2B': 0.9727678874